# 1. Access the element of a dataframe



In [1]:
from pyspark.sql import SparkSession, DataFrame
import os

In [2]:
local=True
if local:
    spark=SparkSession.builder.master("local[4]").appName("RemoveDuplicates").getOrCreate()
else:
    spark=SparkSession.builder \
                      .master("k8s://https://kubernetes.default.svc:443") \
                      .appName("RemoveDuplicates") \
                      .config("spark.kubernetes.container.image","inseefrlab/jupyter-datascience:master") \
                      .config("spark.kubernetes.authenticate.driver.serviceAccountName",os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
                      .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
                      .config("spark.executor.instances", "4") \
                      .config("spark.executor.memory","8g") \
                      .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1') \
                      .getOrCreate()                

2022-11-09 11:01:56,682 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1.1 Get column as a list

This use case is useful when you have a categorical column, and you want to get the distinct value of the colum as a list

In [3]:
data = [("James", "Sales", 3000),
            ("Michael", "Sales", 4600),
            ("Robert", "Sales", 4100),
            ("Maria", "Finance", 3000),
            ("James", "Sales", 3000),
            ("Scott", "Finance", 3300),
            ("Jen", "Finance", 3900),
            ("Jeff", "Marketing", 3000),
            ("Kumar", "Marketing", 2000),
            ("Saif", "Sales", 4100)
            ]
columns = ["name", "department", "salary"]
df = spark.createDataFrame(data=data, schema=columns)
print("Source data frame: ")
df.printSchema()
df.show(truncate=False)
df.count()

Source data frame: 
root
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



+-------+----------+------+
|name   |department|salary|
+-------+----------+------+
|James  |Sales     |3000  |
|Michael|Sales     |4600  |
|Robert |Sales     |4100  |
|Maria  |Finance   |3000  |
|James  |Sales     |3000  |
|Scott  |Finance   |3300  |
|Jen    |Finance   |3900  |
|Jeff   |Marketing |3000  |
|Kumar  |Marketing |2000  |
|Saif   |Sales     |4100  |
+-------+----------+------+



10

In [8]:
name=df.select("name").distinct()

In [9]:
print(type(name))

<class 'pyspark.sql.dataframe.DataFrame'>


In [10]:
print(name)

DataFrame[name: string]


In [17]:
name_list=name.toPandas()['name'].tolist()

In [18]:
print(type(name_list))
print(name_list)

<class 'list'>
['James', 'Michael', 'Maria', 'Robert', 'Scott', 'Jen', 'Kumar', 'Saif', 'Jeff']
